In [1]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import mysql.connector

from datetime import datetime
  
# ct stores current time
now = datetime.now()
formatted_date = now.strftime('%Y-%m-%d %H:%M:%S')



db= mysql.connector.connect(

	host = "localhost",
	user = "root",
	passwd = "root",
    database = "fedd"
	

	)
mycursor = db.cursor()


face_classifier = cv2.CascadeClassifier(r'A:\sem6project\Emotion_Detection_CNN-main\haarcascade_frontalface_default.xml')
classifier =load_model(r'A:\sem6project\Emotion_Detection_CNN-main\model.h5')

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)




while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)



        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            mycursor.execute('INSERT INTO data (emotion, DT) VALUES (%s,%s)',(label,formatted_date))
            
            
            db.commit()
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
print(prediction)

[0.17869583 0.02064851 0.15866484 0.08786819 0.18069902 0.35725766
 0.01616597]


In [3]:
print(label)

Sad


In [4]:
print(label_position)

(201, 88)


In [5]:
print(frame)

[[[ 79  96 101]
  [ 81  98 103]
  [ 82  99 106]
  ...
  [129 156 160]
  [134 156 158]
  [134 156 158]]

 [[ 82  97 100]
  [ 83  98 101]
  [ 84  99 103]
  ...
  [132 158 163]
  [132 159 161]
  [132 159 161]]

 [[ 82  97 100]
  [ 84  99 103]
  [ 85 100 105]
  ...
  [134 158 161]
  [135 160 162]
  [134 158 161]]

 ...

 [[ 92  95  92]
  [ 97 100  97]
  [ 95 100  97]
  ...
  [ 68  68  68]
  [ 68  66  68]
  [ 66  63  66]]

 [[ 93  96  94]
  [ 98 101  98]
  [ 96 101 100]
  ...
  [ 69  68  70]
  [ 68  66  68]
  [ 64  61  63]]

 [[ 97  97  95]
  [100 101  98]
  [101 102  99]
  ...
  [ 69  68  72]
  [ 68  67  69]
  [ 65  65  67]]]


In [4]:
print(ct)

2022-02-03 18:23:31.684808


In [3]:
print(dt)

2022-02-03 18:23:31
